In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings # to get emb
from langchain.vectorstores import DeepLake  # vector store
from langchain.text_splitter import CharacterTextSplitter #split large content based on char count
from langchain import OpenAI
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import SeleniumURLLoader 
from langchain import PromptTemplate

In [3]:
urls = ['https://beebom.com/what-is-nft-explained/',
    'https://beebom.com/how-delete-spotify-account/',
    'https://beebom.com/how-download-gif-twitter/',
    'https://beebom.com/how-use-chatgpt-linux-terminal/',
    'https://beebom.com/how-delete-spotify-account/',
    'https://beebom.com/how-save-instagram-story-with-music/',
    'https://beebom.com/how-install-pip-windows/',
    'https://beebom.com/how-check-disk-usage-linux/']

### Split the documents into chunks and compute their embeddings and save in DB


In [4]:
loader = SeleniumURLLoader(urls=urls)
raw_docs = loader.load()
print(len(raw_docs))

# split the raw docs into chunks based on chunk size
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print(len(docs))



8
104


In [5]:
# get embeddings and store them in the Deep lake 
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

my_activeloop_org_id = "charnvardhan"
my_activeloop_database_name = "LangChainCustomerSupport"
dataset_path = f"hub//{my_activeloop_org_id}/{my_activeloop_database_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

db.add_documents(docs)

Creating 104 embeddings in 1 batches of size 104:: 100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

Dataset(path='hub//charnvardhan/LangChainCustomerSupport', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (104, 1)      str     None   
 metadata     json      (104, 1)      str     None   
 embedding  embedding  (104, 1536)  float32   None   
    id        text      (104, 1)      str     None   


['b5d95ae2-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95bd2-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95c04-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95c2c-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95c4a-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95c68-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95c90-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95cae-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95ccc-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95cea-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d08-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d26-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d44-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d62-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d80-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95d9e-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95dbc-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95dda-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95df8-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95e16-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95e34-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95e52-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95e70-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95e8e-14f0-11f0-a1e8-0e8d4396c6f5',
 'b5d95eac-14f0-

In [12]:
# Retrieve the most relevant document.
query = "how to check the disk usage in Linux"
docs = db.similarity_search(query) # k to get top k from the similar docs
print(docs[0].page_content)

Home > Tech > How to Check Disk Usage in Linux (4 Methods)

How to Check Disk Usage in Linux (4 Methods)

Beebom Staff

Updated: December 19, 2023

Comments 0

Share

Copied

There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.

Table of Contents

Check Disk Space Using the df Command

In Linux, there are many commands to check disk usage, the most common being the df command. The df stands for “Disk Filesystem” in the command, which is a handy way to check the current disk usage and the available disk space in Linux. The syntax for the df command in Linux is as follows:

df <options> <file_system>

In [13]:
len(docs)

4

## Craft a prompt for using the suggested strategies

In [14]:
template = """You are an exceptional customer support chathot that gently answer questions.

you know the follwing context information.

{chunks_formatted}

Answer the following question from a customer. Use only information from the previous context information.
Do not invent stuff.

question: {query}

Answer:
"""

In [15]:
prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template
)

## Generate answers from the LLM

In [23]:
# The pipeline

query = "How to check disk usage in Linux"

#retrieve relevant chunks
docs = db.similarity_search(query, k=2)
print(len(docs))
retrieved_chunks = [doc.page_content for doc in docs]

#format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_format = prompt.format(chunks_formatted=chunks_formatted, query=query)

print(prompt_format)

2
You are an exceptional customer support chathot that gently answer questions.

you know the follwing context information.

Home > Tech > How to Check Disk Usage in Linux (4 Methods)

How to Check Disk Usage in Linux (4 Methods)

Beebom Staff

Updated: December 19, 2023

Comments 0

Share

Copied

There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.

Table of Contents

Check Disk Space Using the df Command

In Linux, there are many commands to check disk usage, the most common being the df command. The df stands for “Disk Filesystem” in the command, which is a handy way to check the current disk u

In [26]:
# generate answer
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=prompt_format)]
answer = llm(messages)
print(answer)

content='To check disk usage in Linux, you can use the df command in the terminal. The df command stands for "Disk Filesystem" and allows you to check the current disk usage and available disk space on your system. Additionally, you can also use GUI tools like the GDU Disk Usage Analyzer and the Gnome Disks Tool to easily monitor disk usage in a more user-friendly way. These tools can be installed using the provided commands.' additional_kwargs={} example=False
